In [ ]:
import sinaps as sn

# Reaction Diffusion

In this example we will have cacium and buffer...

## Initialization

Creating a sample neuron:

In [ ]:
nrn = sn.Neuron([(0,1),(1,2),(1,3),(3,4)])
nrn.plot()

Settings channels: note that the HH_Ca channel is to be used in additon to a HH channel to add the calcium dynamic

In [ ]:
nrn[0].clear_channels()#Reset all channels, useful if you re-run this cell (does anything the first time)
nrn[0].add_channel(sn.channels.HeavysideCurrent(500,2,3),0)
nrn[0].add_channel(sn.channels.Hodgkin_Huxley())
nrn[0].add_channel(sn.channels.Hodgkin_Huxley_Ca(gCa=14.5,V_Ca=115)) 

In [ ]:
sim = sn.Simulation(nrn,dx=10)

In [ ]:
sim.run((0,100))

In [ ]:
sim.plot()

## Species

We define shortcuts of species:

In [ ]:
#shortcuts
Ca=sn.Species.Ca
BF=sn.Species.Buffer
BFB=sn.Species.Buffer_Ca

## Chemical reactions

In [ ]:
#Reset reaction / not useful the first time
nrn.reactions=[]

We model the chemical equation
$$ Ca + Bf \rightleftharpoons_{k_2}^{k_1} BfCa $$

Note that Species are automatically added to the neuron if they are not already
present.

In [ ]:
nrn.add_reaction(
    {Ca:1,
     BF:1},
    {BFB:1},
    k1=0.2,
    k2=0.1)

In [ ]:
#Calcium extrusion
nrn.add_reaction(
    {Ca:1},
    {},
    k1=0.05,k2=0)

## Initial concentrations

In [ ]:
#Calcium initial concentration
for s in nrn.sections:
    s.C0[Ca]=10**(-7)

## Running electro diffusion simulatiom

In [ ]:
sim.run_diff(max_step=1)

## Plotting results

In [ ]:
sim.plot.C(Ca)

In [ ]:
sim.plot.C(BFB)

In [ ]:
(sim.plot.V().opts(width=900,height=200) 
 + sim.plot.C(Ca) .opts(width=900,height=200) 
 + sim.plot.C(BFB).opts(width=900,height=200) 
).cols(1).opts(plot={'shared_axes':False})

In [ ]:
sim.plot.C_field(Ca)